## Direct location with Sentinel-1 SAR sensor 

This notebook shows examples on direct location performed using SAR sensor and using DEM GETAS zarr format.

In [ ]:
import os

import numpy as np

from asgard.core.time import JD_TO_SECONDS
from asgard.models.body import EarthBody
from asgard.models.sar import SPEED_LIGHT
from asgard.models.time import TimeReference
from asgard.sensors.sentinel1.csar import S1SARGeometry
from asgard_legacy_drivers.drivers.sentinel_1_legacy import S1LegacyDriver

### Setup variables and configure S1SARGeometry

In [ ]:
# these directories are required and must be set beforehand
TEST_DIR = os.environ.get("TEST_DIR", "../tests")
ASGARD_DATA = os.environ.get("ASGARD_DATA", "/data/asgard")
GETAS_PATH = os.path.join(ASGARD_DATA, "ADFstatic/S0__ADF_GETAS_20000101T000000_21000101T000000_20240529T142617.zarr")

In [ ]:
# Instanciate time model
iers_data = S1LegacyDriver.read_iers_file(os.path.join(TEST_DIR, "resources", "207_BULLETIN_B207.txt"))
time_model = TimeReference(iers_bulletin_b=iers_data)
driver = S1LegacyDriver(EarthBody(time_reference=time_model))
tref = driver.time_reference

# azimuth times for each burst taken from a s1a_ew1_slc-*.xml product
swath_azimuth_times_ew1 = [
    "2022-11-11T11:47:01.513329",
    "2022-11-11T11:47:04.552211",
    "2022-11-11T11:47:07.591093",
    "2022-11-11T11:47:10.629975",
    "2022-11-11T11:47:13.665937",
    "2022-11-11T11:47:16.704819",
    "2022-11-11T11:47:19.746620",
    "2022-11-11T11:47:22.779664",
    "2022-11-11T11:47:25.815627",
    "2022-11-11T11:47:28.857428",
    "2022-11-11T11:47:31.893391",
    "2022-11-11T11:47:34.935192",
    "2022-11-11T11:47:37.974074",
    "2022-11-11T11:47:41.010037",
    "2022-11-11T11:47:44.051838",
    "2022-11-11T11:47:47.087800",
    "2022-11-11T11:47:50.123763",
    "2022-11-11T11:47:53.162645",
]

# data exracted from EW3 (in the middle of the swath)
range_sampling_rate_ew3 = 2.502314816000000e07
range_sampling_time_ew3 = 1 / range_sampling_rate_ew3
ew1_slant_range_time_ew3 = 5.556849666167215e-03
ew1_samples_ew3 = 8457

# Convert swath E1 azimuth times to TX
azimuth_zd_times = np.array(
    [tref.from_str(item, unit="s", epoch="2022-11-11T11:47:00") for item in swath_azimuth_times_ew1],
    dtype="float64",
)
azimuth_tx_times = azimuth_zd_times - 0.5 * (ew1_slant_range_time_ew3 + ew1_samples_ew3 * 0.5 * range_sampling_time_ew3)

ew1_azimuth = {"offsets": azimuth_tx_times, "ref": "UTC", "unit": "s", "epoch": "2022-11-11T11:47:00"}


# azimuth times for each burst taken from a s1A_ew2_slc-*.xml product
swath_azimuth_times_ew2 = [
    "2022-11-11T11:47:02.140955",
    "2022-11-11T11:47:05.176918",
    "2022-11-11T11:47:08.215800",
    "2022-11-11T11:47:11.254682",
    "2022-11-11T11:47:14.290644",
    "2022-11-11T11:47:17.332446",
    "2022-11-11T11:47:20.368408",
    "2022-11-11T11:47:23.407290",
    "2022-11-11T11:47:26.443253",
    "2022-11-11T11:47:29.482135",
    "2022-11-11T11:47:32.521017",
    "2022-11-11T11:47:35.559899",
    "2022-11-11T11:47:38.598781",
    "2022-11-11T11:47:41.637663",
    "2022-11-11T11:47:44.673625",
    "2022-11-11T11:47:47.712507",
    "2022-11-11T11:47:50.751389",
    "2022-11-11T11:47:53.790271",
]

# same data exracted from EW3 (in the middle of the swath)
range_sampling_rate_ew3 = 2.502314816000000e07
range_sampling_time_ew3 = 1 / range_sampling_rate_ew3
ew2_slant_range_time_ew3 = 5.556849666167215e-03
ew2_samples_ew3 = 8457

# Convert swath E2 azimuth times to TX
azimuth_zd_times_ew2 = np.array(
    [tref.from_str(item, unit="s", epoch="2022-11-11T11:47:00") for item in swath_azimuth_times_ew2],
    dtype="float64",
)
azimuth_tx_times_ew2 = azimuth_zd_times_ew2 - 0.5 * (
    ew2_slant_range_time_ew3 + ew2_samples_ew3 * 0.5 * range_sampling_time_ew3
)

# use days unit for EW2
azimuth_tx_times_ew2 *= 1 / JD_TO_SECONDS

ew2_azimuth = {"offsets": azimuth_tx_times_ew2, "ref": "UTC", "unit": "d", "epoch": "2022-11-11T11:47:00"}


# Exract orbit informations, use RESORB instead of PREORB+PVT as they give a poor accuracy
orbit_file = os.path.join(
    TEST_DIR,
    "resources",
    "S1",
    "RESORB",
    "S1A_OPER_AUX_RESORB_OPOD_20221111T145817_V20221111T111732_20221111T143502.EOF",
)

resorb_20221111 = driver.read_orbit_file(orbit_file)

# Extract attitude informations
attitude_file = os.path.join(
    TEST_DIR,
    "resources",
    "S1",
    "PVT_ATT_20221111T114656",
    "s1_attitude_eocfi.xml",
)
att_20221111 = driver.read_attitude_file(attitude_file)

# Configuration JSON example for S1SARGeometry with swath EW1 and EW2
config = {
    "sat": "SENTINEL_1A",
    "look_side": "RIGHT",
    "swaths": {
        "EW1": {
            "azimuth_times": ew1_azimuth,
            "azimuth_convention": "TX",
            "azimuth_time_interval": 2.919194958309765e-03,
            "burst_lines": 1168,
            "slant_range_time": 4.969473533235427e-03,
            "range_sampling_rate": 2.502314816000000e07,
            "burst_samples": 8337,
        },
        "EW2": {
            "azimuth_times": ew2_azimuth,
            "azimuth_convention": "TX",
            "azimuth_time_interval": 2.919194958309765e-03,
            "burst_lines": 1163,
            "slant_range_time": 5.293253736597336e-03,
            "range_sampling_rate": 2.502314816000000e07,
            "burst_samples": 6824,
        },
    },
    "oper_mode": "EW",
    "orbits": [resorb_20221111],
    "attitude": att_20221111,
    "resources": {
        "dem_path": GETAS_PATH,
        "dem_type": "ZARR_GETAS",
    },
    "eop": {"iers_bulletin_b": iers_data},
}

sar_geometry = S1SARGeometry(**config)

# Perform direct locatino on EW2
img_coords = np.array(
    [
        [0, 0],
        [342, 0],
        [684, 0],
        [0, 1163],
        [342, 1163],
        [684, 1163],
    ],
    dtype="int64",
)
gnd_ew2, times_ew2 = sar_geometry.direct_loc(img_coords, geometric_unit="EW2", altitude=0.0)


# Apply inverse location on last direct loc results for EW2
sar_geometry.propagation_model.config["terrain_height_lut"] = {
    "azimuth": {
        "offsets": times_ew2,
        "unit": "s",
        "epoch": ew2_azimuth["epoch"],
        "ref": "UTC",
    },
    "height": gnd_ew2[:, 2],
}
pixels_ew2 = sar_geometry.inverse_loc(gnd_ew2[:, :2], altitude=gnd_ew2[:, 2], geometric_unit="EW2")


# Prepare data to compute slant range localisation on EW1
azimuth_times = {
    "offsets": np.array(
        [tref.from_str(item, unit="s", epoch="2022-11-11T11:47:00") for item in swath_azimuth_times_ew1[:6]],
        dtype="float64",
    ),
    "ref": "UTC",
    "unit": "s",
    "epoch": "2022-11-11T11:47:00",
}

# Slant range times taken from image annotation of s1a-ew1-slc-*.xml
range_times = np.array(
    [
        4.969473533235427e-03,
        4.986138103070272e-03,
        5.002802672905118e-03,
        4.969473533235427e-03,
        4.986138103070272e-03,
        5.002802672905118e-03,
    ],
    dtype="float64",
)
range_distance = range_times * (SPEED_LIGHT * 0.5)

altitudes = np.array(
    [
        8.109956979751587e-04,
        7.661003619432449e-04,
        7.254164665937424e-04,
        8.110590279102325e-04,
        7.662279531359673e-04,
        7.255943492054939e-04,
    ],
    dtype="float64",
)

# Compute slant range localisation
gnd, gnd_velocity = sar_geometry.slant_range_localisation(
    azimuth_times,
    range_distance,
    altitudes,
    compute_velocity=True,
)

dataset = {"position": gnd}
driver.earth_body.cartesian_to_geodetic(dataset)

gnd_slant_range_ew1 = dataset["position"]


# Prepare data to compute terrain height on EW1

# Terrain height times taken from general annotation of s1a-ew1-slc-*.xml
terrain_azimuth_str = [
    "2022-11-11T11:47:11.513329",
    "2022-11-11T11:47:21.513329",
    "2022-11-11T11:47:31.513329",
    "2022-11-11T11:47:41.513329",
]

terrain_azimuth = [tref.from_str(item, unit="s", epoch="2022-11-11T11:47:00") for item in terrain_azimuth_str]

# Compute of terrain height for swath EW1
elevations = sar_geometry.terrain_height(
    terrain_azimuth,
    azimuth_block_size=1,
    azimuth_subsampling=1,
    range_subsampling=100,
    geometric_unit="EW1",
)

In [ ]:
elevations